* Find file location on disk
  * Place under Git
  * Create new IHE-SDC Git repo for SDC Object Model Usage Samples
  * Create a folder for SDC XML files
  * Create "Assmblies" folder for imported assemblies
    * Copy in the latest SDC.Schema dll 
    * Copy in "C:\Program Files\Microsoft Visual Studio\2022\Enterprise\Common7\IDE\PublicAssemblies\Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll" //Microsoft.VisualStudio.TestTools.UnitTesting
    * Create Archive folder for junk notebooks

In [ ]:
//Import assemblies from files
#r "./Assemblies/SDC_CodeGeneratorTest.dll"  //SDC.Schema namespace
#r "./Assemblies/Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll"  //unit test framework, if we want to use Asserts etc.

using System;
using SDC.Schema;
//
    //using SDC;
    //using System.Runtime;
    //using System.IO;

    using Microsoft.VisualStudio.TestTools.UnitTesting;
    //using Newtonsoft.Json;
    //using System.Data;
    //using System.Diagnostics;
    //using System.Drawing;
    //using System.Linq;
    //using System.Xml; 

* Read in a large SDC file, and then deserialize as XML

In [ ]:
string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize
string myXML =  SdcSerializer<FormDesignType>.Serialize(FD); //Serialize
//myXML

## Create iterator to return nodes

In [ ]:
using System.Diagnostics;
//------------------------------------------------------------------------
////SDC Setup
    BaseType.ResetSdcImport();
    string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
    var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize

////TIMER SETUP
    Stopwatch.StartNew();
    var timerStartTime  = (float)Stopwatch.GetTimestamp();
    display(timerStartTime);
//---------------------------------------------------------------------
int i = 0;

foreach (BaseType node in NodeIterator(FD))
    {
        //btPrint(node);
        //Handle old-new node comparisons here
    }
///////////////////////////////////////////////////
IEnumerable<BaseType> NodeIterator(BaseType? n)
    {
        while (n is not null)
        {
            n = MoveNext(n);
            if (n is not null)
                yield return n;
            else yield break;
        }
        yield break;
    }
/////////////////////////////////////////////////
BaseType? MoveNext(BaseType n)
{
    Dictionary<Guid, List<BaseType>> cn = n.TopNode.ChildNodes;
    List<BaseType>? childList;
    BaseType? nextNode;
    
    n.order = i;  //almost instananeous
    Assert.IsTrue(n.ObjectID == i);//very fast
    i++;
    //if n has child nodes, the next node is the first child node of n.
    if (cn.TryGetValue(n.ObjectGUID, out childList))
    {
        nextNode = childList[0];
        if (nextNode is not null) return nextNode;
    }
    //Notes:
        //n has no child nodes here, so walk up the tree to the parent node.
        //When we walk back up the object graph, prevPar is the original starting node (deeper in the tree),
        //while par is the parent of prevPar.  par is more superficial in the tree, and closer to the top node
        //We then check the childList of par, to see if prePar can be found in that childList
        //If prevPar is in childList, then try to retrieve the next node in childList
        //IF we cant get the nextNode from childList, then move up to one higher parent level
    var prevPar = n;  
    var par = prevPar.ParentNode;

    while (par is not null)
    {
        if (cn.TryGetValue(par.ObjectGUID, out childList))
        {
            var index = childList.IndexOf(prevPar);
            if (index < childList.Count - 1)
            {
                nextNode = childList[index + 1];
                if (nextNode is not null) return nextNode;
            }
            //the next node is not located yet, so walk up to a previous ancestor and try again,
            //looking in that ancestors childList for a nextNode candidate
            prevPar = par; 
            par = prevPar.ParentNode;
        }
    }
    return null;
}
/////////////////////////Small method to output node info///////////////////////////////
void btPrint(BaseType n)
    {
        string content;
        if (n is not null)
        {
            if (n is DisplayedType) content = ": title: " + (n as DisplayedType)?.title;
            else if (n is PropertyType) content = ": " + (n as PropertyType)?.propName + ": " + (n as PropertyType)?.val;
            else content = "";

            display(n.ObjectID.ToString().PadLeft(4) + ": " + i.ToString().PadLeft(4) + ": " + (n.name ?? "").PadRight(20) + ": " + (n.ElementName ?? "").PadRight(25) + content);
        }
    }
display(Stopwatch.GetTimestamp());
display(((Stopwatch.GetTimestamp() - timerStartTime) / Stopwatch.Frequency).ToString());

1.05675063E+12

1056750672279

0.0065536

## Find SDC node changes compared to the reference template
* Create 2 FormDesign OMs, "newOM" and "refOM"
* Iterate each node in newOM, and locate matcching node in refOM by ID, SGuid and/or name
  * For each newOM node, determine if it's 
    * new, 
    * has a different parent node, 
    * previous sib has changed.
  * Add the changed node's ID/name/sGUID and change type to a dictionary
* Compare each SDC property in newOM with the matching refOM node
* 

In [ ]:
// var a1 = new { A = 1, B = 2, C = 3, D = 4, E = 5 };
// var a2 = a1 with { E = 10 }; 
// Console.WriteLine (a2);      // { A = 1, B = 2, C = 3, D = 4, E = 10 }
using System.Linq;

public readonly record struct nDiff (BaseType nOld, string ChangedPropName , string ChangedPropVal);

bool BaseTypeChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        
        if (nOld.name != nNew.name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.name), nOld.name));}
        if (nOld.type != nNew.type) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.type), nOld.type));}
        if (nOld.styleClass != nNew.styleClass) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.styleClass), nOld.styleClass));}
        if (nOld.sGuid != nNew.sGuid) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.sGuid), nOld.sGuid));}
        if (nOld.order != nNew.order) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.order), nOld.order.ToString()));}

        if (nOld.ParentNode.name != nNew.ParentNode.name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "ParentNode.name", nOld.ParentNode.name));}
        if (nOld.ParentIETypeNode.name != nNew.ParentIETypeNode.name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "ParentIETypeNode.name", nOld.ParentIETypeNode.name));}
        if (nOld.ParentIETypeID != nNew.ParentNode.ParentIETypeID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.ParentIETypeID), nOld.ParentIETypeID));}
        
        //Check previous sibling match, if present
        var prevSibOld = nOld.GetNodePreviousSib();
        var prevSibNew = nOld.GetNodePreviousSib();
        if(prevSibOld.name != prevSibNew.name)
        {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "PreviousSib.name", prevSibOld?.name??""));}

        //Check node type
        if (nOld.GetType().Name != nNew.GetType().Name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "GetType.Name", nOld.GetType().Name));}
        //Check parent node type
        if (nOld.ParentNode.GetType().Name != nNew.ParentNode.GetType().Name) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "ParentNode.GetType.Name", nOld.ParentNode.ParentNode.GetType().Name));}
        //Check all Properties, Comments, Extensions here?

        if (changed) return false;            

        return true;
    }
    bool QuestionChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = RepeatTypeChanged(nOld, nNew, nChanges);

        //QuestionItemBaseType
        if (nOld.readOnly != nNew.readOnly) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.readOnly), nOld.readOnly.ToString()));}
        if (nOld.changedData != nNew.changedData) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.changedData), nOld.changedData.ToString()));}
        if (nOld.newData != nNew.newData) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.newData), nOld.newData.ToString()));}
        if (nOld.GetQuestionSubtype() != nNew.GetQuestionSubtype()) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "QuestionSubtype", nOld.GetQuestionSubtype().ToString()));}
        
        if (!SpecialPropertyChanged(nOld, nNew, nChanges)) changed = false;

        if (nOld.GetQuestionSubtype() == QuestionEnum.QuestionFill)
        {
            //Look for some LIR properties, like datatype, minInclusive, maxInclusive, fractionDigits, val, quantEnum, etc.
            //Surface any changes, so they appear in this node
            //Also TextAfterResponse, Response Units

            //Retrieve DataTypes_DEtype object (Response Element)
            DataTypes_DEType respOld = nOld.ResponseField_Item?.Response;
            DataTypes_DEType respNew = nOld.ResponseField_Item?.Response;

            string respTextAfterOld = nOld.ResponseField_Item?.TextAfterResponse?.val;
            string respTextAfterNew = nNew.ResponseField_Item?.TextAfterResponse?.val;
            string respUnitsOld = nOld.ResponseField_Item?.ResponseUnits?.val;
            string respUnitsNew = nNew.ResponseField_Item?.ResponseUnits?.val;
            string unitSysOld = nOld.ResponseField_Item?.ResponseUnits?.unitSystem;
            string unitSysNew = nNew.ResponseField_Item?.ResponseUnits?.unitSystem;

            string dtOldName = respOld?.DataTypeDE_Item?.NodeType.ToString();
            string dtNewName = respNew?.DataTypeDE_Item?.NodeType.ToString();
            BaseType dtOldNode = respOld?.DataTypeDE_Item;
            BaseType dtNewNode = respOld?.DataTypeDE_Item;            

            Type dtOld = dtOldNode?.GetType();
            Type dtNew = dtOldNode?.GetType();
            //Need to check that dtOld=dtNew, and handle gracefully



            System.Reflection.FieldInfo[] fieldsOld = dtOld?.GetFields(); //TODO: only retrieve serializable fields via attributes?
            System.Reflection.FieldInfo[] fieldsNew = dtNew?.GetFields(); //TODO: only retrieve serializable fields via attributes?

            dynamic maxIncOld = fieldsOld?.Where(f=>f.Name=="maxInclusive")?.First()?.GetValue(dtOldNode); //this may not work, since we don't know the type at compile time
            dynamic maxIncNew = fieldsNew?.Where(f=>f.Name=="maxInclusive")?.First()?.GetValue(dtNewNode); //Conside dynamic type or converting results to string

            dynamic minIncOld = fieldsOld?.Where(f=>f.Name=="minInclusive")?.First()?.GetValue(dtOldNode);
            dynamic minIncNew = fieldsNew?.Where(f=>f.Name=="minInclusive")?.First()?.GetValue(dtNewNode);

            dynamic maxExOld = fieldsOld?.Where(f=>f.Name=="maxExclusive")?.First()?.GetValue(dtOldNode); //this may not work, since we don't know the type at compile time
            dynamic maxExNew = fieldsNew?.Where(f=>f.Name=="maxExclusive")?.First()?.GetValue(dtNewNode);
            dynamic minExOld = fieldsOld?.Where(f=>f.Name=="minExclusive")?.First()?.GetValue(dtOldNode);
            dynamic minExNew = fieldsNew?.Where(f=>f.Name=="minExclusive")?.First()?.GetValue(dtNewNode);

            byte? fracDigOld = fieldsOld?.Where(f=>f.Name=="fractionDigits")?.First()?.GetValue(dtOldNode) as byte?;
            byte? fracDigNew = fieldsNew?.Where(f=>f.Name=="fractionDigits")?.First()?.GetValue(dtNewNode) as byte?;
            dynamic valOld = fieldsOld?.Where(f=>f.Name=="val")?.First()?.GetValue(dtOldNode);
            dynamic valNew = fieldsNew?.Where(f=>f.Name=="val")?.First()?.GetValue(dtNewNode);

            byte? totDigOld = fieldsOld?.Where(f=>f.Name=="totalDigits")?.First()?.GetValue(dtOldNode) as byte?;
            byte? totDigNew = fieldsNew?.Where(f=>f.Name=="totalDigits")?.First().GetValue(dtNewNode) as byte?;

            byte? minLenOld = fieldsOld?.Where(f=>f.Name=="minLength")?.First()?.GetValue(dtOldNode) as byte?;
            byte? minLenNew = fieldsNew?.Where(f=>f.Name=="minLength")?.First()?.GetValue(dtNewNode) as byte?;

            byte? maxLenOld = fieldsOld?.Where(f=>f.Name=="maxLength")?.First()?.GetValue(dtOldNode) as byte?;
            byte? maxLenNew = fieldsNew?.Where(f=>f.Name=="maxLength")?.First()?.GetValue(dtNewNode) as byte?;

            string? maskOld = fieldsOld?.Where(f=>f.Name=="mask")?.First()?.GetValue(dtOldNode) as string;
            string? maskNew = fieldsNew?.Where(f=>f.Name=="mask")?.First()?.GetValue(dtNewNode) as string;

            string? patternOld = fieldsOld?.Where(f=>f.Name=="pattern")?.First()?.GetValue(dtOldNode) as string;
            string? patternNew = fieldsNew?.Where(f=>f.Name=="pattern")?.First()?.GetValue(dtNewNode) as string;

            string? quantEnumOld = fieldsOld?.Where(f=>f.Name=="quantEnum")?.First()?.GetValue(dtOldNode) as string;
            string? quantEnumNew = fieldsNew?.Where(f=>f.Name=="quantEnum")?.First()?.GetValue(dtNewNode) as string;

            //TODO:
                //allowGT
                //allowGTE
                //allowLT
                //allowLTE
                //allowAPPROX

        }
        if (nOld.GetQuestionSubtype() == QuestionEnum.QuestionSingleOrMultiple)
        {
            //Surface minSelections, maxSelections, numCols... changes to this Question node
        }

        return changed;
    }

    bool SpecialPropertyChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;

        if(!ReportTextPropertyChanged(nOld, nNew, nChanges)) changed = true;
        if(!AltTextPropertyChanged(nOld, nNew, nChanges)) changed = true;

        return changed;
    }

    bool PropertyChanged(PropertyType nOld, PropertyType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        if(BaseTypeChanged(nOld, nNew, nChanges)) changed = true;
        if (nOld.propName != nNew.propName) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.propName), nOld.propName));}
        if (nOld.propClass != nNew.propClass) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.propClass), nOld.propClass));}
        if (nOld.val != nNew.val) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.val), nOld.val));}
        
        return changed;
    }
    bool PropertyRepeatsChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
        {
            bool changed = false;
            //Group all Properties by propName, for both nOld and nNew,
            //so that we can determine if some property nodes occur more than once, or if the counts differ between nOld and nNew
            //We are using nNew as teh basis for comparison, such that any property nodes that were removed from nOld will not show up here
            //-------------------------
            var pNew =
            from p in nNew.GetChildList().OfType<PropertyType>().ToList()
            group p by p.propName into propsByPropName
            select propsByPropName;
            //------------------------
            var pOld =
            from p in nNew.GetChildList().OfType<PropertyType>().ToList()
            group p by p.propName into propsByPropName
            select propsByPropName;
            //------------------------

            foreach (IGrouping<string, PropertyType> propsByPropNameNew in pNew)
            {
                //iterate through the propName-grouped Properties in pNew
                foreach (IGrouping<string, PropertyType> propsByPropNameOld in pOld.Where(p=> propsByPropNameNew.Key ==p.Key))
                {//find propName-grouped Properties in pOld (i.e., the current IGrouping named propsByPropNameOld) where the propName matches the current IGrouping from pNew (propsByPropNameNew)
                    //i.e., where propsByPropNameNew.Key ==p.Key

                    var newCount = propsByPropNameNew.Count();
                    var oldCount = propsByPropNameNew.Count();
                    //If the counts for the current propName differ in pOld and pNew, flag it as a change:
                    if (newCount != oldCount) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "Counts_OldPropName_NewPropName", oldCount.ToString() + " - " +  newCount.ToString()));}
                }

                //Search for propNames in pOld that are not found in pNew

            }
            
            return changed;
        }


    bool ReportTextPropertyChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        //reportText
        List<PropertyType> newReportText = nNew.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "reportText")?.ToList();
        List<PropertyType> oldReportText = nOld.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "reportText")?.ToList();
        if (newReportText[0] !=oldReportText[0]) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "reportText", oldReportText[0]?.ToString()));}  

        return changed;
    }
    bool AltTextPropertyChanged(BaseType nOld, BaseType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        //reportText
        List<PropertyType> newAltText = nNew.GetChildList().OfType<PropertyType>()?.Where(p=>p.propName == "altText")?.ToList();
        List<PropertyType> oldAltText = nOld.GetChildList().OfType<PropertyType>().Where(p=>p.propName == "altText")?.ToList();
        if (newAltText[0] !=oldAltText[0]) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, "altText", oldAltText[0]?.ToString()));}
        //(oldAltText[0] is null)?"":oldAltText[0].val)
        return changed;
    }
    bool PropertyChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
          

        return changed;
    }
    bool FormDesignHeaderPropertyChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = BaseTypeChanged(nOld, nNew, nChanges);
        //Assess FormDesign attributes
        //Assess Properties         

        return changed;
    }

    bool LIRDatatypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = BaseTypeChanged(nOld, nNew, nChanges);
        //Assess the datatype element
        //Assess Properties         

        return changed;
    }
    bool QuestionDatatypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = false;
        //Assess the datatype element
        //Assess Properties         

        return changed;
    }
    bool RepeatTypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = DisplayedTypeChanged(nOld, nNew, nChanges);
                  
        //RepeatType
        if (nOld.minCard != nNew.minCard) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.minCard), nOld?.minCard.ToString()));}
        if (nOld.maxCard != nNew.maxCard) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.maxCard), nOld?.maxCard.ToString()));}
        if (nOld.repeat != nNew.repeat) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.repeat), nOld?.repeat.ToString()));}
        if (nOld.instanceGUID != nNew.instanceGUID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.instanceGUID), nOld?.instanceGUID));}
        if (nOld.parentGUID != nNew.parentGUID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.parentGUID), nOld?.parentGUID));}

        return changed;
    }

    bool DisplayedTypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = IdentifiedExtensionTypeChanged(nOld, nNew, nChanges);

        //DisplayedType
        if (nOld.title != nNew.title) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.title), nOld.title));}
        if (nOld.enabled != nNew.enabled) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.enabled), nOld?.enabled.ToString()));}
        if (nOld.visible != nNew.visible) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.visible), nOld.visible.ToString()));}
        if (nOld.mustImplement != nNew.mustImplement) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.mustImplement), nOld.mustImplement.ToString()));}
        if (nOld.showInReport != nNew.showInReport) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.showInReport), nOld.showInReport.ToString()));}

        return changed;
    }
    bool IdentifiedExtensionTypeChanged(QuestionItemType nOld, QuestionItemType nNew, Dictionary<string, nDiff> nChanges)
    {
        bool changed = BaseTypeChanged(nOld, nNew, nChanges);
        
        //IET
        if (nOld.ID != nNew.ID) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.ID), nOld.ID));}
        if (nOld.baseURI != nNew.baseURI) {changed = true; nChanges.Add(nOld.name, new nDiff(nOld, nameof(nOld.baseURI), nOld.baseURI));}

        return changed;
    }



Error: (57,13): error CS0246: The type or namespace name 'DataTypeDEType' could not be found (are you missing a using directive or an assembly reference?)
(58,13): error CS0246: The type or namespace name 'DataTypeDEType' could not be found (are you missing a using directive or an assembly reference?)

In [ ]:
var bt = new FormDesignType();

if(null == null) display(true);

int i = 123;
object o = new object();
object o2;
display(o.GetType().Name);
o= i;

display(o.ToString());
display(o.GetType().Name);
display(o2.ToString());

True

Object

123

Int32

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#52.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)